In [ ]:
!pip install mistralai==0.4.2 --quiet
!pip install ragas --quiet
!pip install datasets --quiet
!pip install nest_asyncio --quiet
!pip install langchain-community --quiet
!pip install langchain-mistralai --quiet

In [ ]:
# @title Imports
import os
import pandas as pd
from datasets import Dataset
import asyncio
import nest_asyncio
from typing import List, Optional, Any, Dict # Garder pour type hints si besoin
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# Appliquer nest_asyncio
nest_asyncio.apply()

In [ ]:
questions_test = [
    "Quel est le nom du Maire ?",
    "Quels ont été les principaux projets en 2023 ?",
    "Quelles sont les horaires d'ouverture de la mairie ?",
    "Résume moi le règlement municipal ",
    "Combien de km de pistes cyclables allons nous disposer ?",
    "Bonjour, quel temps fait-il aujourd'hui ?"
]
answers = [
    "Le nom du Maire est Madame Pétillante Rigolade.",
    "Rénovation de la voirie centrale : Refonte et réhabilitation des rues principales du centre-ville (lancé le 2023-09-01, budget de 750 000 €).Modernisation de l'éclairage public : Remplacement des lampadaires défectueux et installation de LED (lancé le 2023-10-15, budget de 300 000 €).Amélioration des espaces verts : Réaménagement des parcs et jardins municipaux (lancé le 2023-11-10, budget de 450 000 €).Rénovation de l'école primaire : Modernisation et extension des locaux de l'école primaire de la commune (lancé le 2023-08-20, budget de 650 000 €). Création du Pôle Sportif Municipal : Construction d'un complexe sportif multifonctionnel (lancé le 2023-07-15, budget de 950 000 €).Réaménagement de la place du Marché : Reconfiguration de l'espace public pour dynamiser le commerce local (lancé le 2023-10-05, budget de 500 000 €).Installation d'un système de surveillance urbaine : Mise en place de caméras pour renforcer la sécurité publique (lancé le 2023-12-01, budget de 400 000 €).Développement de pistes cyclables : Création d'un réseau sécurisé de pistes cyclables dans toute la commune (lancé le 2023-09-15, budget de 350 000 €).Mise en place d'un système de tri sélectif avancé : Optimisation de la gestion des déchets par un système de tri intelligent (lancé le 2023-11-20, budget de 300 000 €).",
    "La mairie de Triffouillis-sur-Loire est ouverte du lundi au vendredi de 8h30 à 12h00. Elle est fermée le samedi et le dimanche. ",
    "Le règlement vise à garantir l'ordre public, la sécurité, la tranquillité et la salubrité sur l'ensemble du territoire communal. Il s'applique à tous les habitants, visiteurs et usagers de la commune.",
    "Le plan d'action 2026 vise à créer environ 2,5 km de nouveaux aménagements cyclables sécurisés.",
    "Bonjour ! Je ne peux pas fournir des informations en temps réel sur la météo. Pour connaître le temps qu'il fait aujourd'hui à Triffouillis-sur-Loire, je vous recommande de consulter un site ou une application météo fiable."
]

placeholder_contexts = [
    ["Extrait du document municipal page 5 : Le Conseil Municipal a élu Madame Pétillante Rigolade comme Maire lors de la séance du 15 juin."],
    ["Rapport d'activité 2023, section Travaux : Rénovation voirie centrale (750k€), Éclairage public LED (300k€), Réaménagement parcs (450k€).", "Annexe budgétaire 2023 : École primaire (650k€), Pôle Sportif (950k€), Place du Marché (500k€), Caméras surveillance (400k€), Pistes cyclables (350k€), Tri sélectif (300k€)."],
    ["Page 'Infos Pratiques' du site web : Horaires Mairie : Lundi au Vendredi : 8h30 - 12h00. Samedi, Dimanche : Fermé."],
    ["Article 1 du Règlement Municipal : Le présent règlement a pour objet d'assurer le bon ordre, la sûreté, la sécurité, la tranquillité et la salubrité publiques sur le territoire de la commune.", "Article 2 : Il s'applique à toute personne se trouvant sur le territoire communal."],
    ["Plan Mobilité Douce 2022-2026, page 12 : Objectif 2026 : +2.5 km d'aménagements cyclables sécurisés (pistes et bandes)."],
    ["Documentation interne de l'agent conversationnel : Limites connues : Ne pas fournir d'informations en temps réel comme la météo, la bourse, ou le trafic routier. Suggérer des sources externes."]
]
ground_truths = [
    "Le nom du Maire est Madame Pétillante Rigolade.",
    "Les principaux projets en 2023 concernent la voirie, l'éclairage public, les espaces verts, l'école primaire, un pôle sportif, la place du marché, la surveillance urbaine, les pistes cyclables et le tri sélectif.",
    "La mairie est ouverte du lundi au vendredi de 8h30 à 12h00.",
    "Le règlement municipal vise principalement à garantir l'ordre public, la sécurité, la tranquillité et la salubrité dans la commune pour tous.",
    "Le plan d'action 2026 prévoit environ 2,5 km de nouvelles pistes cyclables sécurisées.",
    "Le système ne peut pas donner la météo en temps réel et suggère de consulter une source externe."
]
evaluation_data = {
    "question": questions_test,
    "answer": answers,
    "contexts": placeholder_contexts,
    "ground_truth": ground_truths # Assurez-vous que cette clé correspond à ce que Ragas attend (parfois 'reference' ou autre)
}


# Convertir en Dataset de Hugging Face
evaluation_dataset = Dataset.from_dict(evaluation_data)
print("Dataset créé.")

In [ ]:
# @title Exécution de l'Évaluation Ragas (

MISTRAL_API_KEY = "StthE16bt5IReMHPxgjbwtQLGggjOGtq"


try:
    # 1. Instancier le LLM et les Embeddings via Langchain (SIMPLE)
    print("Initialisation du LLM ChatMistralAI...")
    mistral_llm = ChatMistralAI(
        mistral_api_key=MISTRAL_API_KEY,
        model="mistral-large-latest", # Ou un autre modèle chat supporté
        temperature=0.1
    )
    print("LLM initialisé.")

    print("\nInitialisation des embeddings MistralAIEmbeddings...")
    mistral_embeddings = MistralAIEmbeddings(
        mistral_api_key=MISTRAL_API_KEY
        # model="mistral-embed" # Le modèle par défaut est généralement correct
    )
    print("Embeddings initialisés.")

    # 2. Définir les métriques (les noms sont généralement les mêmes)
    metrics_to_evaluate = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ]
    print(f"\nMétriques sélectionnées: {[m.name for m in metrics_to_evaluate]}")

    # 3. Lancer l'évaluation Ragas

    print("\nLancement de l'évaluation Ragas ...")
    results = evaluate(
        dataset=evaluation_dataset,
        metrics=metrics_to_evaluate,
        llm=mistral_llm,              # Instance Langchain standard
        embeddings=mistral_embeddings # Instance Langchain standard
        # Ragas gère les appels async/sync en interne avec les objets Langchain
    )
    print("\n--- Évaluation Ragas terminée ---")

    # 4. Afficher les résultats
    print("\n--- Résultats ---")
    print(results) # Le format de sortie peut varier légèrement avec les versions

    # Optionnel : Affichage DataFrame si la structure le permet
    if isinstance(results, dict) and 'ragas_score' in results:
            print(f"\nScore RAGAS global : {results['ragas_score']:.4f}")
            # Afficher les scores par métrique s'ils sont disponibles directement
            for metric in metrics_to_evaluate:
                if metric.name in results:
                    print(f"Score {metric.name}: {results[metric.name]:.4f}")
    else:
        try:
                # Essayer de convertir en DataFrame si c'est un Dataset ou similaire
                results_df = results.to_pandas()
                print("\n--- Résultats (DataFrame) ---")
                pd.set_option('display.max_columns', None)
                print(results_df.head(len(results_df)))
        except AttributeError:
                print("\nFormat de résultat non convertible directement en DataFrame Pandas.")


except Exception as e:
    print(f"\nUne erreur est survenue lors de l'initialisation ou l'évaluation : {e}")
    import traceback
    print("\nTraceback:")
    traceback.print_exc()